<a href="https://colab.research.google.com/github/siddharthchd/deepLearning_20/blob/main/GANs/fashionMNIST_DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from PIL import Image

(x_train, _), (x_test, _) = tf.keras.datasets.fashion_mnist.load_data()

In [7]:
batch_size = 64
images = np.concatenate([x_train, x_test])
images = images / 255.
images = np.reshape(images, (-1, 28, 28, 1))

dataset = tf.data.Dataset.from_tensor_slices(images)
dataset = dataset.shuffle(buffer_size = 1024).batch(batch_size).prefetch(32)

In [ ]:
x_test.shape

(10000, 28, 28)

In [20]:
def discriminator():

  model = tf.keras.Sequential([
      
      tf.keras.layers.Conv2D(64, kernel_size = 5, strides= 2, padding = 'same', input_shape = (28, 28, 1)),
      #tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.LeakyReLU(0.3),
      tf.keras.layers.Dropout(0.3),           

      tf.keras.layers.Conv2D(128, kernel_size = 5, strides= 2, padding = 'same'),
      #tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.LeakyReLU(0.3),
      tf.keras.layers.Dropout(0.3), 

      tf.keras.layers.GlobalMaxPooling2D(),
      #tf.keras.layers.Flatten(),
      #tf.keras.layers.Dense(512),
      #tf.keras.layers.LeakyReLU(0.3),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])

  return model

In [21]:
discriminator = discriminator()
discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
global_max_pooling2d (Global (None, 128)              

In [86]:
def generator(gauss_size):

  model = tf.keras.Sequential([
      tf.keras.layers.Dense(7 * 7 * 256, input_dim = gauss_size),
      tf.keras.layers.Reshape((7, 7, 256)),
      
      tf.keras.layers.Conv2DTranspose(256, 5, padding = 'same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.3),

      tf.keras.layers.Conv2DTranspose(128, 5, strides = 1, padding = 'same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.3),

      tf.keras.layers.Conv2DTranspose(64, 5, strides = 2, padding = 'same'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.LeakyReLU(0.3),

      tf.keras.layers.Conv2DTranspose(1, 2, strides = 2, activation = 'tanh')
  ])

  return model

In [87]:
generator = generator(100)
generator.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
reshape_28 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_76 (Conv2DT (None, 7, 7, 256)         1638656   
_________________________________________________________________
batch_normalization_63 (Batc (None, 7, 7, 256)         1024      
_________________________________________________________________
leaky_re_lu_96 (LeakyReLU)   (None, 7, 7, 256)         0         
_________________________________________________________________
conv2d_transpose_77 (Conv2DT (None, 7, 7, 128)         819328    
_________________________________________________________________
batch_normalization_64 (Batc (None, 7, 7, 128)       

In [89]:
class GAN(tf.keras.Model):

  def __init__(self, discriminator, generator, latent_dim):

    super(GAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim

  def compile(self, discriminator_opt, generator_opt, loss):

    super(GAN, self).compile()
    self.discriminator_opt = discriminator_opt
    self.generator_opt = generator_opt
    self.loss = loss

  def train_step(self, images):

    images = images[0]
    batch_size = tf.shape(images)[0]
    random_vecs = tf.random.normal(shape = (batch_size, self.latent_dim))

    generated_images = self.generator(random_vecs)
    combined_images=  tf.concat([generated_images, images], axis = 0)

    labels = tf.conact([tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis = 0)
    labels += 0.05 * tf.random.uniform(tf.shape(labels))

    with tf.GradientTape() as tape:

      predictions = self.discriminator_opt(combined_images)
      discriminator_loss = self.loss(labels, predicitions)

    gradients = tape.gradient(discriminator_loss, self.discriminator.trainable_weights)
    self.discriminator_opt.apply_gradients(zip(gradients, self.discriminator.trainable_weights))

    radom_vecs = tf.random.normal(shape = (batch_size, self.latent_dim))

    misleading_labels = tf.zeros((batch_size, 1))

    with tf.GradientTape() as tape:

      predictions = self.discriminator(self.generator(random_vecs))
      generator_loss = self.loss(misleading_labels, predictions)
      gradients = tape.gradient(generator_loss, self.generator.trainable_weights)
      self.generator_opt.apply_gradients(zip(gradients, self.generator.trainable_weights))

      return {'discriminator_loss' : discriminator_loss, 'generator_loss' : generator_loss}